In [333]:
import pandas as pd
import string
import re

# from helper_functions.google_books_api import fetch_genre_from_google_books

### Read Books, Ratings and Users csv file

In [334]:
df_books = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/raw/Books.csv", sep=';', engine='python')
df_ratings = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/raw/Ratings.csv", sep=';', engine='python')
df_users = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/raw/Users.csv", sep=';', engine='python')


In [335]:
df_books.shape

(271379, 5)

In [336]:
df_ratings.shape

(1149780, 3)

In [337]:
df_users.shape

(278859, 2)

In [338]:
# Rename books columns
df_books.columns = ['isbn', 'title', 'author', 'year', 'publisher']

# Rename ratings columns
df_ratings.columns = ['user_id', 'isbn', 'rating']

# Rename users columns
df_users.columns = ['user_id', 'age']


In [339]:
# Convert to string first, then clean
df_ratings = df_ratings[df_ratings['user_id'].astype(str).str.strip().str.isnumeric()]
df_users = df_users[df_users['user_id'].astype(str).str.strip().str.isnumeric()]

# Now convert to integer
df_ratings['user_id'] = df_ratings['user_id'].astype(int)
df_users['user_id'] = df_users['user_id'].astype(int)


### Matching Data Types

In [340]:
# Merge ratings with books on 'isbn'
df_merged = pd.merge(df_ratings, df_books, on='isbn', how='inner')

# Merge the result with users on 'user_id's
df_final = pd.merge(df_merged, df_users, on='user_id', how='inner')


In [341]:
df_final.to_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/books_dataset.csv")

In [342]:
df_final.describe(include='all')

,user_id,isbn,rating,title,author,year,publisher,age
count,1.031187e+06,1031187,1.031187e+06,1031187,1031185,1.031187e+06,1031185,753677
unique,NaN,270170,NaN,241090,101594,NaN,16832,189
top,NaN,0971880107,NaN,Wild Animus,Stephen King,NaN,Ballantine Books,33
freq,NaN,2502,NaN,2502,10053,NaN,34724,32740
mean,1.405945e+05,NaN,2.839018e+00,NaN,NaN,1.968194e+03,NaN,NaN
std,8.052442e+04,NaN,3.854146e+00,NaN,NaN,2.311035e+02,NaN,NaN
min,2.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN
25%,7.041500e+04,NaN,0.000000e+00,NaN,NaN,1.992000e+03,NaN,NaN
50%,1.412100e+05,NaN,0.000000e+00,NaN,NaN,1.997000e+03,NaN,NaN
75%,2.114260e+05,NaN,7.000000e+00,NaN,NaN,2.001000e+03,NaN,NaN


In [343]:
df_final.columns

Index(['user_id', 'isbn', 'rating', 'title', 'author', 'year', 'publisher',
       'age'],
      dtype='object')

In [344]:
# Find duplicated ISBNs (same book listed multiple times)
duplicate_isbns = df_books[df_books.duplicated('isbn', keep=False)]

# Count frequency of each duplicate ISBN
duplicate_isbn_counts = duplicate_isbns.groupby('isbn').size().reset_index(name='count')
duplicate_isbn_counts = duplicate_isbn_counts.sort_values(by='count', ascending=False)

# Display top duplicates
print(duplicate_isbn_counts.head(10))


         isbn  count
0  0486404242      2


In [345]:

# Normalize titles: lowercase, strip, remove punctuation
df_final['normalized_title'] = df_final['title']\
    .str.lower()\
    .str.strip()\
    .apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Count unique normalized titles
unique_title_count = df_final['normalized_title'].nunique()
print(f"Unique normalized titles: {unique_title_count}")


Unique normalized titles: 236507


In [346]:
# Extract unique normalized titles
unique_titles = df_final['normalized_title'].dropna().unique()

# Save to CSV
import pandas as pd
pd.DataFrame(unique_titles, columns=['normalized_title']).to_csv('unique_titles.csv', index=False)


In [347]:
df_final.shape

(1031187, 9)

In [348]:
best_books = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/raw/best_books.csv")

In [349]:
best_books.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1


In [350]:
# Extract title and genres
title_genres_df = best_books[['title', 'genres']]

# Save to CSV file
title_genres_df.to_csv('/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/title_genres.csv', index=False)


In [351]:
best_books = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/unique_titles.csv")
unique_title_with_genres = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/title_genres.csv")

In [352]:
best_books.columns

Index(['normalized_title'], dtype='object')

In [353]:
unique_title_with_genres.columns

Index(['title', 'genres'], dtype='object')

In [354]:

# # Normalize function
# def normalize(text):
#     return re.sub(r'\W+', '', str(text).lower().strip())

# # Use 'title' instead of 'normalized_title'
# unique_title_with_genres['normalized_title_clean'] = unique_title_with_genres['title'].apply(normalize)
# best_books['normalized_title_clean'] = best_books['title'].apply(normalize)

# # Mapping
# genre_map = dict(zip(best_books['normalized_title_clean'], best_books['genres']))

# # Add matched genres
# unique_title_with_genres['matched_genres'] = None

# for i, row in unique_title_with_genres.iterrows():
#     current_title = row['normalized_title_clean']
#     for norm_title, genres in genre_map.items():
#         if norm_title in current_title or current_title in norm_title:
#             unique_title_with_genres.at[i, 'matched_genres'] = genres
#             break

# # Save
# unique_title_with_genres.to_csv('/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/enriched_titles_with_genres.csv', index=False)

# print("✅ Enriched file saved as 'enriched_titles_with_genres.csv'")


In [355]:
enriched_titles_with_genres = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/enriched_titles_with_genres.csv")

In [356]:
enriched_titles_with_genres.head(5)

,title,genres,normalized_title_clean,matched_genres
0,The Hunger Games,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",thehungergames,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas..."
1,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",harrypotterandtheorderofthephoenix,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',..."
2,To Kill a Mockingbird,"['Classics', 'Fiction', 'Historical Fiction', ...",tokillamockingbird,"['Classics', 'Fiction', 'Historical Fiction', ..."
3,Pride and Prejudice,"['Classics', 'Fiction', 'Romance', 'Historical...",prideandprejudice,"['Classics', 'Fiction', 'Romance', 'Historical..."
4,Twilight,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...",twilight,"['Fiction', 'Holocaust', 'Jewish', 'Historical..."


In [357]:
enriched_titles_with_genres.shape

(52478, 4)

In [358]:
enriched_titles_with_genres.isna().sum()

title                     0
genres                    0
normalized_title_clean    0
matched_genres            0
dtype: int64

In [359]:
df_final.head(5)

,user_id,isbn,rating,title,author,year,publisher,age,normalized_title
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,NaN,flesh tones a novel
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,NaN,rites of passage
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,16,the notebook
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,16,help level 1
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,16,the amsterdam connection level 4 cambridge en...


In [360]:

# Merge on normalized_title
df_final = df_final.merge(
    enriched_titles_with_genres[['normalized_title_clean', 'matched_genres']],
    how='left',
    left_on='normalized_title',
    right_on='normalized_title_clean'
)

# Drop helper column
df_final.drop(columns='normalized_title_clean', inplace=True)

# Save result
df_final.to_csv('/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/df_final_with_genres.csv', index=False)

In [361]:
data_with_no_null_genre = df_final[~df_final['matched_genres'].isna()]

In [362]:
# Divide rating by 2
data_with_no_null_genre['rating'] = data_with_no_null_genre['rating'] / 2

# Remove rows where rating is NaN or 0
data_with_no_null_genre = data_with_no_null_genre[
    data_with_no_null_genre['rating'].notna() & (data_with_no_null_genre['rating'] != 0)
]

/var/folders/dw/fn12lc3x2b317ydc5b70v2340000gn/T/ipykernel_46134/1830435904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_no_null_genre['rating'] = data_with_no_null_genre['rating'] / 2


In [363]:
data_with_no_null_genre.to_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/data_with_no_null_genre.csv")

In [364]:
good_reads = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/raw/good_read_books_100k.csv")

In [365]:
good_reads.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings'],
      dtype='object')

In [366]:
good_reads.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [367]:
data_with_no_null_genre.head()

,user_id,isbn,rating,title,author,year,publisher,age,normalized_title,matched_genres
190,276850,0449217264,3.5,Alaska,James A. Michener,1994,Fawcett Books,16,alaska,"['Young Adult', 'Fiction', 'Contemporary', 'Ro..."
262,276872,0425167097,4.0,Toxin,Robin Cook,1999,Berkley Pub Group,43,toxin,[]
263,276872,042518109X,4.0,Whispers,Dean R. Koontz,2001,Berkley Publishing Group,43,whispers,"['Fiction', 'Romance', 'Drama', 'Family', 'Nov..."
264,276872,042518109X,4.0,Whispers,Dean R. Koontz,2001,Berkley Publishing Group,43,whispers,"['Fiction', 'Romance', 'Drama', 'Family', 'Nov..."
265,276872,042518109X,4.0,Whispers,Dean R. Koontz,2001,Berkley Publishing Group,43,whispers,"['Fiction', 'Romance', 'Drama', 'Family', 'Nov..."


In [368]:
data_with_no_null_genre.columns

Index(['user_id', 'isbn', 'rating', 'title', 'author', 'year', 'publisher',
       'age', 'normalized_title', 'matched_genres'],
      dtype='object')

In [369]:
good_reads.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings'],
      dtype='object')

In [377]:

# Convert genre from string to list if necessary
def parse_genre(val):
    if pd.isna(val):
        return []
    if isinstance(val, list):
        return val
    return [g.strip() for g in str(val).split(',') if g.strip()]

good_reads['matched_genres'] = good_reads['genre'].apply(parse_genre)

# Create a unified format for good_reads
good_reads_standardized = pd.DataFrame({
    'user_id': data_with_no_null_genre['user_id'],
    'isbn': good_reads['isbn'],
    'rating': good_reads['rating'],
    'title': good_reads['title'],
    'author': good_reads['author'],
    'desc':good_reads['desc'],
    'year': data_with_no_null_genre['year'],
    'publisher': data_with_no_null_genre['publisher'],
    'age': data_with_no_null_genre['age'],
    'normalized_title': good_reads['title'].str.lower().str.replace(r'[^a-z0-9]', '', regex=True),
    'matched_genres': good_reads['matched_genres']
})

# Select and align columns from data_with_no_null_genre
data_with_no_null_genre_aligned = data_with_no_null_genre[[
    'user_id', 'isbn', 'rating', 'title', 'author', 'year', 'publisher',
    'age', 'normalized_title', 'matched_genres'
]]

# Combine the two datasets
combined_df = pd.concat([data_with_no_null_genre_aligned, good_reads_standardized], ignore_index=True)

# Save the result
combined_df.to_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/combined_books.csv", index=False)

print("✅ Combined and standardized dataset saved as combined_books.csv")

✅ Combined and standardized dataset saved as combined_books.csv


In [378]:
combined_df = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/combined_books.csv")


/var/folders/dw/fn12lc3x2b317ydc5b70v2340000gn/T/ipykernel_46134/3982326125.py:1: DtypeWarning: Columns (1,3,4,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/combined_books.csv")


In [379]:
combined_df.shape

(151774, 11)